# Spam Filter

In this project, we're going to build a **spam filter for SMS messages** using the multinomial Naive Bayes algorithm. Our goal is to write a program that classifies new messages with an **accuracy greater than 80%** — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

To train the algorithm, we'll use a dataset of 5,572 SMS messages that are already classified by humans. The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the <a href="https://archive.ics.uci.edu/ml/datasets/sms+spam+collection">The UCI Machine Learning Repository</a>. The data collection process is described in more details on <a href="https://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition">this page</a>, where you can also find some of the papers authored by Tiago A. Almeida and José María Gómez Hidalgo.

# Requiremets

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

# Exploring the dataset

In [3]:
df = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

In [4]:
df

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   5572 non-null   object
 1   SMS     5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [6]:
df.describe()

,Label,SMS
count,5572,5572
unique,2,5169
top,ham,"Sorry, I'll call later"
freq,4825,30


In [7]:
df["Label"].value_counts(normalize=True) * 100

ham     86.593683
spam    13.406317
Name: Label, dtype: float64

From the previous list, we can see that about **87%** of the messages are ham, and the remaining **13%** are spam. This sample looks representative, since in practice most messages that people receive are ham.

# Training and test sets

In [8]:
data_randomized = df.sample(frac=1, random_state=1)
train_test_portion = round(len(data_randomized) * 0.8)

training_set = data_randomized[:train_test_portion].reset_index(drop=True)
test_set = data_randomized[train_test_portion:].reset_index(drop=True)

We'll now analyze the percentage of spam and ham messages in the training and test sets. We expect the percentages to be close to what we have in the full dataset, where about 87% of the messages are ham, and the remaining 13% are spam.

Training set shape and percentages:

In [9]:
training_set.shape

(4458, 2)

In [10]:
training_set["Label"].value_counts(normalize=True) * 100

ham     86.54105
spam    13.45895
Name: Label, dtype: float64

Test set shape and percentages:

In [12]:
test_set.shape

(1114, 2)

In [11]:
test_set["Label"].value_counts(normalize=True) * 100

ham     86.804309
spam    13.195691
Name: Label, dtype: float64

The results look good! We'll now move on to cleaning the dataset.

# Data cleaning

To calculate all the probabilities required by the algorithm, we'll first need to perform a bit of data cleaning to bring the data in a format that will allow us to extract easily all the information we need.

In [13]:
training_set

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired
3,ham,Havent.
4,ham,I forgot 2 ask ü all smth.. There's a card on ...
...,...,...
4453,ham,"Sorry, I'll call later in meeting any thing re..."
4454,ham,Babe! I fucking love you too !! You know? Fuck...
4455,spam,U've been selected to stay in 1 of 250 top Bri...
4456,ham,Hello my boytoy ... Geeee I miss you already a...


### Letter case & punctuation

We start by removing all the punctuation and bringing converting every letter to lower case.

In [14]:
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ')
training_set['SMS'] = training_set['SMS'].str.lower()
training_set.head()

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...


### Creating Vocabulary

In [15]:
training_set['SMS'] = training_set['SMS'].str.split()

vocabulary = []
for sms in training_set['SMS']:
    for word in sms:
        vocabulary.append(word)
vocabulary = list(set(vocabulary))            

In [16]:
len(vocabulary)

7783

It looks like there are 7,783 unique words in all the messages of the training set.

### Counting words per sms

In [21]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

In [22]:
for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

### Final training set

In [23]:
training_set = pd.concat([training_set, pd.DataFrame(word_counts_per_sms)], axis=1)
training_set.head()

,Label,SMS,soup,kb,doke,webadres,sf,um,requests,cr9,...,nickey,ldnw15h,ranjith,restrictions,ruthful,imprtant,newest,lil,reltnship,overtime
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
